In [8]:
#importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [9]:
df=pd.read_csv('energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
#to check if there is any column with missing data
df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [10]:
#droping data nd light columns
df.drop(['lights','date'],axis=1, inplace= True)

In [11]:
#scaling the variables in the dataset to give them same range.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
normalized_df=pd.DataFrame(scaler.fit_transform(df),columns=df.columns)
features_df=normalized_df.drop(columns=['Appliances'])
target=normalized_df['Appliances']

In [13]:
#Question 12
#running a simple linear regresssion betwee column T2 and T6
x=np.asanyarray(normalized_df[['T2']])
y=np.asanyarray(normalized_df[['T6']])
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)
lm=LinearRegression()
lm.fit(x_train,y_train)
lm.score(x_test,y_test)
predic_lm=lm.predict(x_test)



In [14]:
#Question 12
# The R^2 score
rsquared=(r2_score(y_test,predic_lm))
print(round(rsquared,2))

0.64


In [15]:
#Question 13
#calculating the mean absolute error
mae=mean_absolute_error(y_test,predic_lm)
print(round(mae,2))

0.08


In [19]:
#Question 14
rss=np.sum(np.square(y_test-predic_lm))
print(round(rss,2))

55.14


In [18]:
#Question 15
#rmse
rmse=np.sqrt(mean_squared_error(y_test,predic_lm))
print(round(rmse,3))

0.106


In [37]:
# A multivariate regression model. To answer question 17-20
x_train,x_test,y_train,y_test=train_test_split(features_df,target,test_size=0.3,random_state=42)


In [38]:
lm2=LinearRegression() 
lm2.fit(x_train,y_train)
lm2.score(x_test,y_test)
predic_lm=lm2.predict(x_test)

In [39]:
#question 17
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(lm2, x_train, 'linear_model_weight')
print(linear_model_weights.max())
print(linear_model_weights.min())

Features                    rv2
linear_model_weight    0.553547
dtype: object
Features               Press_mm_hg
linear_model_weight      -0.456698
dtype: object


In [42]:
#ridge
# Using a ridge model 
from sklearn.linear_model import Ridge
ridge=Ridge(alpha=0.4)
ridge.fit(x_train,y_train)
ridge.score(x_test,y_test)
predic_ridge=ridge.predict(x_test)
rmse=np.sqrt(mean_absolute_error(y_test,predic_ridge))
print(rmse)

0.22380224717576866


In [45]:
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.001,max_iter=1000 )
lasso.fit(x_train,y_train)
lasso.score(x_test,y_test)
predic_lasso=lasso.predict(x_test)
print('r2 score on  training set is',lasso.score(x_train,y_train))
print('r2 score on  test set is',lasso.score(x_test,y_test))
print('mean absolute_error on predicted_values is : ',mean_absolute_error(y_test,predic_lasso), 'and' 'r2 score is ',r2_score(y_test,predic_lasso))
print('number of features used= ',np.sum(lasso.coef_!=0))

r2 score on  training set is 0.02499234354320623
r2 score on  test set is 0.026800880567125818
mean absolute_error on predicted_values is :  0.055256639821262235 andr2 score is  0.026800880567125818
number of features used=  4


In [46]:
rmse=np.sqrt(mean_squared_error(y_test,predic_lasso))
print(round(rmse,3))

0.094
